# crawling_naver cafe_게시판 글 목록

### Step 0. 준비

In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
import math

### Step 1. 크롤링

In [2]:
keyword = "☰ 통합 Q & A ☰"
crawling_no = int(input('클롤링 할 글 개수를 입력 :'))

클롤링 할 글 개수를 입력 :10


In [3]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"./chromedriver/chromedriver.exe")

# 사이트 주소
driver.get("https://cafe.naver.com/noljatravel")
time.sleep(2)

# 게시판 클릭
driver.find_element_by_link_text(keyword).click()

# 게시판 프레임 접근
driver.switch_to.frame("cafe_main")

# 게시글 50개씩
driver.find_element_by_css_selector("#listSizeSelectDiv").click()
driver.find_element_by_xpath("/html/body/div[1]/div/div[3]/div/div[3]/ul/li[7]/a").click()

In [4]:
#crawling_list = []
no_app = []
title_app = []
nick_app  = []
like_app  = []

# 크롤링 해야 할 페이지 계산
crawling_page = int(math.ceil(crawling_no / 50)+1)

try: 
    for page in range(1,crawling_page):
        # 페이지 클릭
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
        # 글 번호 수집
        no = [i.text for i in driver.find_elements_by_css_selector('.td_article')]
        no_split = [ni.split()[0] for ni in no]
        # 글 제목 수집
        title = [i.text for i in driver.find_elements_by_css_selector('.article')]
        # 작성자 수집
        nick = [i.text for i in driver.find_elements_by_css_selector('.p-nick .m-tcol-c')]
        # 좋아요 수집
        like = [i.text for i in driver.find_elements_by_css_selector('.td_likes')]
        # 수집 데이터 append
        no_app.append(no_split)
        title_app.append(title)
        nick_app.append(nick)
        like_app.append(like)
        # 10페이지 마다 프린트 & 다음 페이지로 클릭
        if str(page)[-1] == '0':
            print(int(page), 'page 크롤링 완료')
            driver.find_element_by_link_text('다음').click()
# 더이상 페이지가 존재하지 않을 시
except:
    print('더이상 페이지가 존재하지 않음')

driver.close()
    
# 리스트안 리스트 분해
no_list = sum(no_app, [])
title_list = sum(title_app, [])
nick_list = sum(nick_app, [])
like_list = sum(like_app, [])

# 판다스화
df = pd.DataFrame({'번호':no_list,
                   '제목':title_list,
                   '작성자':nick_list,
                   '좋아요':like_list})
# 필독, 공지 삭제
df = df.drop(df[df['번호'] == '필독'].index)
df = df.drop(df[df['번호'] == '공지'].index)
df = df.reset_index(drop=True)

print('글 ', len(df), '개 크롤링 완료. \n크롤링 종료.', sep='')

글 50개 크롤링 완료. 
크롤링 종료.


In [5]:
df

,번호,제목,작성자,좋아요
0,327461,블박 ssd 로 저장하시는 분들은 오류가 없나요?,세종T뚝딱망치,0
1,327460,"P > D, D > P 변경 시 외부 소음이 어느정도 들리시나요?",송파구T내무부장관,0
2,327448,넷플릭스,부산T대저촌놈,0
3,327431,[충전기질문] 단독주택 집밥 완속충전기 설치 하신분?,서울T화야,0
4,327423,[충전질문] 충전 매일매일 하면 안좋은게 있나요?,경기파주T쭌압바,0
5,327418,[AS질문] 핸들 조향 시 소음이 나네요.,춘천T모3,0
6,327393,분명 하차시에 그냥 280킬로였는데,인천T사고쟁이,0
7,327371,[집밥질문] 부산 사시는분들 집밥 문의 드려요.,울산T화랑,0
8,327363,도와주세요) 충전기 설치 건의 거부.,서울T이가,0
9,327353,[충전질문] 저희 아파트 집밥 가능한 충전기인가요??,강원T팡팡팡이,0


In [6]:
# 저장
df.to_excel('crawler_naver cafe_게시판 {}.xlsx'.format(keyword))